In [28]:
from openai import OpenAI
import pickle
import requests
import measure
from measure import baitness_measure

In [14]:
with open("../file_path.txt", 'r') as file:
    file_path = file.read()

with open(file_path, 'r') as file:
    api_key = file.read()

client = OpenAI(
    api_key=api_key,
)

In [15]:
metrics_functions = {
    "n_words": measure.words_count,
    "cw_percentage": measure.common_words_ratio,
    "capitals_ratio": measure.capital_letters_ratio,
    "capitals_count":measure.capital_words_count,
    "bait_punct": measure.clickbait_punctuation_count,
    "nonbait_punct": measure.nonclickbait_punctuation_count,
    "numbers": measure.numbers_count,
    "2nd_pronouns": measure.pronouns_2nd_person_count,
    "superlatives": measure.superlatives_ratio,
    "speculatives": measure.speculatives_count,
    "bait_words": measure.baiting_words_count,
    "polarity": measure.polarity_score,
    "subjectivity": measure.subjectivity_score,
    "fres": measure.flesch_reading_ease_score,
    "batness_measures": baitness_measure,
}


In [16]:
def return_embeddings_chat(prompt):
    res = requests.post(f"https://api.openai.com/v1/embeddings",
        headers = {
          "Content-Type": "application/json",
          "Authorization": f"Bearer {api_key}"
        },
        json={
          "model": "text-embedding-3-large",
          "dimensions": 1000,
          "input": f"{prompt}"
        }).json()
    return res["data"][0]["embedding"]
        

In [18]:
model_path = '../final_models/RF_1000_dimensions_and_informativeness_measures.pkl'

# Load the model
with open(model_path, 'rb') as rf_file:
    loaded_rf_model = pickle.load(rf_file)

In [19]:
title = "Trump is dead. You wont believe!!"

In [21]:
input_to_model = return_embeddings_chat(title)

In [25]:
for function_name, function in metrics_functions.items():
    input_to_model.append(function(title))

In [37]:
loaded_rf_model.predict_proba([input_to_model])[0][1]

0.56

In [38]:
def get_probability_of_clickbait_title(model_path, title):

    with open(model_path, 'rb') as rf_file:
        loaded_model = pickle.load(rf_file)
        
    input_to_model = return_embeddings_chat(title)

    for function_name, function in metrics_functions.items():
        input_to_model.append(function(title))

    return loaded_model.predict_proba([input_to_model])[0][1]

In [39]:
get_probability_of_clickbait_title('../final_models/RF_1000_dimensions_and_informativeness_measures.pkl', "Trump is dead. You wont believe!!")

0.57

In [ ]:
get_probability_of_clickbait_title('../final_models/RF_1000_dimensions_and_informativeness_measures.pkl', "Trump is dead. You wont believe!!")

In [ ]:
# 19 Dogs Who Are Total Fuzz Buckets, y_train: 1
# x_train: 17 Ways To Have An Amazing Valentine's Day All By Yourself, y_train: 1
# x_train: Ross Kemp: Libya’s Migrant Hell review – a deeply powerful plea, y_train: 0
# x_train: I spent Christmas alone — and got exactly what I wanted, y_train: 1
# x_train: Administration Seeks to Regulate Derivatives, y_train: 0

In [44]:
titles_to_pred = [
    "19 Dogs Who Are Total Fuzz Buckets",
    "17 Ways To Have An Amazing Valentine's Day All By Yourself",
    "Ross Kemp: Libya’s Migrant Hell review – a deeply powerful plea",
    "I spent Christmas alone — and got exactly what I wanted",
    "Administration Seeks to Regulate Derivative",
]

In [45]:
for tit in titles_to_pred:
    res = get_probability_of_clickbait_title('../final_models/RF_1000_dimensions_and_informativeness_measures.pkl', tit)
    print(res)

0.93
0.99
0.29
0.6
0.12


In [6]:
from final_models import get_probability_of_clickbait_title

In [7]:
get_probability_of_clickbait_title('../final_models/RF_1000_dimensions_and_informativeness_measures.pkl', "19 Dogs Who Are Total Fuzz Buckets")

0.93

In [8]:
get_probability_of_clickbait_title('../final_models/XGB_1000_dimensions_and_informativeness_measures.pkl', "19 Dogs Who Are Total Fuzz Buckets")

0.9998246